In [21]:
import pandas as pd
from glob import glob
from tqdm import tqdm_notebook as tqdm
import ast

In [22]:
fns = glob('../tweets/*.csv')
dfs = (pd.read_csv(fn, engine='python') for fn in fns)
df = pd.concat(list(tqdm(dfs, total = len(fns), desc = 'Loading CSV data')), sort = False)

In [23]:
# remove outliers
cleaned = df.copy()
lengths = cleaned.full_text.str.len()
cleaned = cleaned[(50 < lengths) & (lengths < 250)]
cleaned['hashtags'] = cleaned.hashtags.apply(ast.literal_eval).apply(lambda x: ';'.join(list(sorted({y['text'] for y in x}))))
cleaned = cleaned[[
    'full_text', 'id', 'retweet_count', 'hashtags', 'user_name',
    'user_screen_name', 'user_statuses_count', 'user_followers_count', 'user_description',
    'user_location', 'user_created_at', 'q'
]]
cleaned['is_retweet'] = cleaned.full_text.str.startswith('RT ')
cleaned = cleaned.copy().fillna('')
print(f'{100 * cleaned.shape[0] / df.shape[0]:.1f}% of data is clean ({cleaned.shape[0]} tweets)')
# cleaned = cleaned[~cleaned.is_retweet].copy().drop(columns = ['is_retweet'])
# print(f'{100 * cleaned.shape[0] / df.shape[0]:.1f}% of data is clean & original ({cleaned.shape[0]} tweets)')

92.3% of data is clean (100325 tweets)


In [24]:
cleaned['text_cleaned'] = (
    cleaned.full_text.str.replace(
        '#', ' # '
    ).str.replace(
        'https?://[^\s]*', ''# '::'
    ).str.replace(
        r'@[^\s]*', '' #'@@'
    ).str.replace(' +', ' ')
)

In [25]:
cleaned['text_cleaned'].head()

0    RT For context: # China 's govt covered up # C...
1    RT Actress # AnkitaLokhande's Mumbai apartment...
2    RT # CoronavirusOutbreak and Lockdown ke baad ...
3    RT Secunderabad Cantonment Board (SCB) officia...
4    RT KP has notified a committee to look at and ...
Name: text_cleaned, dtype: object

In [26]:
cleaned.to_csv('../cleaned_data.csv', index = False)

In [27]:
! ls -lh ../cleaned_data.csv

-rw-r--r--  1 morris.alper  staff    45M Apr  5 23:07 ../cleaned_data.csv
